In [ ]:
# Pandas에 tqdm 기능 등록 (전처리 진행률 확인용)
tqdm.pandas()

# ==========================================
# 1. DB 연결 및 데이터 로드
# ==========================================
# DB 연결 정보 (본인의 환경에 맞게 수정하세요)
DB_USER = "admin"
DB_PASSWORD = "qkqajrwkrnrnrn9_"
DB_HOST = "musinsa-data.c07kuo6ug98z.us-east-1.rds.amazonaws.com"
DB_PORT = 3306
DB_NAME = "musinsa"

# SQLAlchemy 엔진 생성
db_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)

print("📂 DB에서 데이터 로딩 중...")
# 필요한 컬럼만 가져오기 (브랜드 정보 JOIN 추가)
# Collation 충돌 방지를 위해 COLLATE utf8mb4_unicode_ci 추가
query = """
    SELECT
        p.product_id,
        p.product_name,
        p.img_url
    FROM products as p
"""
image_df = pd.read_sql(query, engine)
print(f"✅ 데이터 로드 완료: {len(df)}건")

In [ ]:
import numpy as np
import pandas as pd
from time import perf_counter

# =====================================================
# 설정
# =====================================================
TARGET_ID = 1345460

IMAGE_EMB_PATH = "/content/drive/MyDrive/top_embedded.npz"

start_total = perf_counter()
print("🚀 cosine similarity 파이프라인 시작")

# =====================================================
# 1. 대표 아이템 인덱스 (df 기준)
# =====================================================
t0 = perf_counter()

target_idx_arr = df.index[df["product_id"] == TARGET_ID]
if len(target_idx_arr) == 0:
    raise ValueError("대표 상품 ID가 df에 없습니다.")

target_idx = target_idx_arr[0]

t1 = perf_counter()
print(f"✅ 대표 아이템 인덱스 확보: {t1 - t0:.3f} sec")

# =====================================================
# 2. 이미지 임베딩 로드 + cosine
# =====================================================
t0 = perf_counter()

img_npz = np.load(IMAGE_EMB_PATH)
img_ids = img_npz["ids"]
img_embs = img_npz["embeddings"]

# 정규화
img_embs = img_embs / np.linalg.norm(img_embs, axis=1, keepdims=True)

img_id_to_idx = {pid: i for i, pid in enumerate(img_ids)}

if TARGET_ID not in img_id_to_idx:
    raise ValueError("대표 상품 ID가 image embedding 파일에 없습니다.")

target_img_vec = img_embs[img_id_to_idx[TARGET_ID]]
cos_image_all = img_embs @ target_img_vec

t1 = perf_counter()
print(f"✅ image cosine 완료 (~{len(img_ids)}개): {t1 - t0:.3f} sec")

# =====================================================
# 3. 텍스트 임베딩 6종 cosine (df 기준)
# =====================================================
t0 = perf_counter()

emb_cols = {
    "cos_product": "product_emb",
    "cos_brand": "brand_emb",
    "cos_style": "style_emb",
    "cos_category": "category_emb",
    "cos_brand_desc": "brand_desc_emb",
    "cos_brand_cat": "brand_cat_emb",
}

result_df = df[["product_id", "product_name", "brand"]].copy()

for cos_col, emb_col in emb_cols.items():
    emb = np.vstack(df[emb_col].values)
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)

    target_vec = emb[target_idx]
    result_df[cos_col] = emb @ target_vec

t1 = perf_counter()
print(f"✅ 텍스트 임베딩 cosine 6종 완료: {t1 - t0:.3f} sec")

# =====================================================
# 4. cos_image 매핑 (없는 상품은 0)
# =====================================================
t0 = perf_counter()

result_df["cos_image"] = result_df["product_id"].map(
    lambda x: cos_image_all[img_id_to_idx[x]] if x in img_id_to_idx else 0.0
)

t1 = perf_counter()
print(f"✅ cos_image 매핑 완료 (없는 상품=0): {t1 - t0:.3f} sec")

# =====================================================
# 5. 대표 아이템 맨 위로 이동
# =====================================================
t0 = perf_counter()

target_row = result_df[result_df["product_id"] == TARGET_ID]
other_rows = result_df[result_df["product_id"] != TARGET_ID]

final_df = pd.concat([target_row, other_rows], ignore_index=True)

t1 = perf_counter()
print(f"✅ 대표 아이템 상단 배치 완료: {t1 - t0:.3f} sec")

# =====================================================
# 총 소요 시간
# =====================================================
end_total = perf_counter()
print(f"🔥 전체 소요 시간: {end_total - start_total:.3f} sec")

# 가중치 적용
weights = {
    "cos_product": 0.15,
    "cos_category": 0.15,
    "cos_style": 0.15,
    "cos_image": 0.75,
    "cos_brand": 0.0,
    "cos_brand_desc": 0.1,
    "cos_brand_cat": 0.1
}

# 가중치 적용 후 점수 계산
final_df = final_df.copy()

final_df["weighted_cosine"] = sum(
    final_df[col] * w for col, w in weights.items()
)

# 대표 상품 분리
rep_row = final_df[final_df["product_id"] == TARGET_ID].copy()
cand_df = final_df[final_df["product_id"] != TARGET_ID].copy()

# top30
top30_df = (
    cand_df
    .sort_values("weighted_cosine", ascending=False)
    .head(30)
)

# 대표 아이템
rep_out = rep_row[[
    "product_id",
    "product_name",
    "brand",
    "weighted_cosine"
]].copy()
rep_out["rank_label"] = "대표"

# top30
top30_out = top30_df[[
    "product_id",
    "product_name",
    "brand",
    "weighted_cosine"
]].copy()
top30_out["rank_label"] = "TOP"

result_df = pd.concat(
    [rep_out, top30_out],
    ignore_index=True
)

# 컬럼의 순서 정리
result_df = result_df[
    [
        "rank_label",
        "product_id",
        "product_name",
        "brand",
        "weighted_cosine"
    ]
]
# 상품id의 리스트화
top30_product_ids = top30_out["product_id"].tolist()



from IPython.display import display, HTML

# ===============================
# 1. 입력 데이터
# ===============================

persona_name = "캐주얼 스트릿"

representative_id = TARGET_ID

top30_product_ids = top30_product_ids

# 카테고리 구조 유지
raw_data = {
    "상의": top30_product_ids
}

# ===============================
# 2. item_map 생성 (image_df 기반)
# ===============================

all_product_ids = [representative_id] + top30_product_ids

filtered_df = image_df[image_df["product_id"].isin(all_product_ids)]

item_map = {
    row["product_id"]: {
        "img_url": row["img_url"],
        "product_name": row["product_name"]
    }
    for _, row in filtered_df.iterrows()
}

# ===============================
# 3. HTML 생성
# ===============================

html_out = f"""
<div style="font-family: sans-serif; padding: 10px;">
    <h2 style="border-bottom: 2px solid #333; padding-bottom: 10px;">
        🎨 Persona: <span style="color: #007bff;">{persona_name}</span>
    </h2>
"""

# ===============================
# 3-1. 대표 아이템
# ===============================

rep_item = item_map.get(representative_id)

if rep_item:
    rep_img = rep_item.get("img_url", "https://via.placeholder.com/150")
    rep_name = rep_item.get("product_name", "상품명 없음")
else:
    rep_img = "https://via.placeholder.com/150?text=No+Data"
    rep_name = "DB 미존재"

html_out += f"""
<h3 style="margin-top: 20px; color: #d63384;">⭐ 대표 아이템</h3>
<div style="display: flex; gap: 15px; flex-wrap: wrap;">
    <div style="
        border: 2px solid #d63384;
        border-radius: 8px;
        padding: 10px;
        width: 140px;
        text-align: center;
        box-shadow: 2px 2px 5px rgba(0,0,0,0.15);
        background: #fff5f8;">

        <div style="
            width: 100%;
            height: 160px;
            overflow: hidden;
            border-radius: 4px;
            margin-bottom: 8px;
            background-color: #f8f9fa;">
            <img src="{rep_img}"
                 style="width: 100%; height: 100%; object-fit: cover;"
                 loading="lazy">
        </div>

        <div style="font-size: 12px; font-weight: bold;">
            ID: {representative_id}
        </div>
        <div style="
            font-size: 11px;
            white-space: nowrap;
            overflow: hidden;
            text-overflow: ellipsis;">
            {rep_name}
        </div>
    </div>
</div>
"""

# ===============================
# 3-2. TOP 30 상품
# ===============================

for category, pids in raw_data.items():
    html_out += f"<h3 style='margin-top: 25px; color: #555;'>🏷️ {category} TOP 30</h3>"
    html_out += "<div style='display: flex; gap: 15px; flex-wrap: wrap;'>"

    for pid in pids:
        item = item_map.get(pid)

        if item:
            img_src = item.get("img_url", "https://via.placeholder.com/150")
            p_name = item.get("product_name", "상품명 없음")
        else:
            img_src = "https://via.placeholder.com/150?text=No+Data"
            p_name = "DB 미존재"

        html_out += f"""
        <div style="
            border: 1px solid #ddd;
            border-radius: 8px;
            padding: 10px;
            width: 140px;
            text-align: center;
            box-shadow: 2px 2px 5px rgba(0,0,0,0.1);
            background: white;">

            <div style="
                width: 100%;
                height: 160px;
                overflow: hidden;
                border-radius: 4px;
                margin-bottom: 8px;
                background-color: #f8f9fa;">
                <img src="{img_src}"
                     style="width: 100%; height: 100%; object-fit: cover;"
                     loading="lazy">
            </div>

            <div style="font-size: 12px; font-weight: bold;">
                ID: {pid}
            </div>
            <div style="
                font-size: 11px;
                white-space: nowrap;
                overflow: hidden;
                text-overflow: ellipsis;">
                {p_name}
            </div>
        </div>
        """

    html_out += "</div>"

html_out += "</div>"

# ===============================
# 4. 렌더링
# ===============================
display(HTML(html_out))

